In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np

# 1) Load the Data
company_list = pd.read_excel('Data (2).xlsx')
company_list['Company'] = company_list['Company'].astype(str).fillna('')

# 2) Create Parent Company Column
def extract_parent_company(value):
    matches = re.findall(r'\(([^)]+)\)', value)
    for match in matches:
        if match not in ['Australia', 'Uk', 'Europe', 'Holdings', 'the', 'Myanmar', 'U.K.', 'Thailand', 'Switzerland','Sales']:
            return match
    return None

company_list['Parent Company'] = company_list['Company'].apply(extract_parent_company)

# 3) Data Cleaning
company_list['Parent Company'] = company_list['Parent Company'].apply(lambda x: x.replace("part of", "").replace("formerly", "").strip() if pd.notnull(x) else x)

# Function to create the matching column based on highest value
def create_matching_column(combined_text, vectorizer, column_name):
    tfidf_matrix = vectorizer.fit_transform(combined_text)
    vectors_a = tfidf_matrix[:len(company_list)]
    vectors_b = tfidf_matrix[len(company_list):]
    similarities = cosine_similarity(vectors_b, vectors_a)

    matching_column = []
    matching_value = []

    for row in similarities:
        highest_index = np.argmax(row)
        highest_value = row[highest_index]
        matching_value.append(highest_value)
        matching_column.append(company_list[column_name].iloc[highest_index])

    return matching_column, matching_value

vectorizer = TfidfVectorizer()

# Load data_cleaned DataFrame
data_cleaned = pd.read_csv('trimmed_data.csv')
data_cleaned['Producer'] = data_cleaned['Producer'].fillna('')

# 4) Process for Creating the Matching Companies Column based on highest value
combined_text_companies = pd.concat([company_list['Company'], data_cleaned['Producer']], ignore_index=True)
matching_companies, matching_values = create_matching_column(combined_text_companies, vectorizer, 'Company')
data_cleaned['Matching Company Highest'] = matching_companies
data_cleaned['Matching Company Highest Value'] = matching_values


# Save the updated data_cleaned DataFrame
data_cleaned.to_csv('matched_results.csv', index=False)


In [ ]:
import os
import openai
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Set the OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")


# Prompt for the company name
company_name = input("Enter the company name: ")
print(f"Searching matches for {company_name} and its subsidiaries")

# Send the prompt to the API, use this prompt for all values in highest matching
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "Data Engineer creating lists to use in an app later. Format the list as a python list always."
        },
        {
            "role": "user",
            "content": f"List all of the subsidiaries, holdings, partners and owners of the company {company_name} as a list in python with no other supporting text. Only the list in a usable format with no line breaks."
        }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# Extract the list of subsidiaries from the response
subsidiaries = eval(response['choices'][0]['message']['content'])
print(f"The listed subsidiaries are {subsidiaries}")
# Load the trimmed_data.csv
data_cleaned = pd.read_csv('trimmed_data.csv')
data_cleaned['Producer'] = data_cleaned['Producer'].fillna('')

# Create a temporary DataFrame using the retrieved subsidiaries
company_temp_df = pd.DataFrame({
    'Company': subsidiaries,
    'Parent Company': [company_name] * len(subsidiaries)
})

import os
import openai
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt


# Initialize the vectorizer
vectorizer = TfidfVectorizer()

def calculate_similarity_matrix(combined_text, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(combined_text)
    vectors_a = tfidf_matrix[:len(company_temp_df)]
    vectors_b = tfidf_matrix[len(company_temp_df):]
    return cosine_similarity(vectors_b, vectors_a)

# Run the similarity calculation process using the retrieved subsidiaries
combined_text_companies = pd.concat([company_temp_df['Company'], data_cleaned['Producer']], ignore_index=True)
similarity_matrix_producers = calculate_similarity_matrix(combined_text_companies, vectorizer)

combined_text_brand = pd.concat([company_temp_df['Company'], data_cleaned['Brand']], ignore_index=True)
similarity_matrix_brands = calculate_similarity_matrix(combined_text_brand, vectorizer)


In [3]:
import os
import openai
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Placeholder: Simulated function to get subsidiaries, holdings, partners, and owners
# Replace this with the actual API call in your environment
# This should be placed in your original code after you calculate 'Matching Company Highest Value'
low_value_matches = data_cleaned[data_cleaned['Matching Company Highest Value'] < 0.10]
print(low_value_matches)
def calculate_similarity_matrix(combined_text, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(combined_text)
    vectors_a = tfidf_matrix[:len(company_temp_df)]
    vectors_b = tfidf_matrix[len(company_temp_df):]
    return cosine_similarity(vectors_b, vectors_a)

def simulated_api_call(company_name):
    openai.api_key = os.environ.get("OPENAI_API_KEY")


# Prompt for the company name
  
    print(f"Searching matches for {company_name} and its subsidiaries")

# Send the prompt to the API, use this prompt for all values in highest matching
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Data Engineer creating lists to use in an app later. Format the list as a python list always."
            },
            {
                "role": "user",
                "content": f"List all of the subsidiaries, holdings, partners and owners of the company {company_name} as a list in python with no other supporting text. Only the list in a usable format with no line breaks."
            }
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

# Extract the list of subsidiaries from the response
    subsidiaries = eval(response['choices'][0]['message']['content'])
    print(f"The listed subsidiaries are {subsidiaries}")
    return subsidiaries

# Initialize an empty list to keep track of updated entries
updated_entries = []

# Loop through each low-value match
for index, row in low_value_matches.iterrows():
    company_name = row['Matching Company Highest']

    # Step 1: Get the new list from the simulated API call
    new_company_list = simulated_api_call(company_name)

    # Step 2: Create a temporary DataFrame
    company_temp_df = pd.DataFrame({
        'Company': new_company_list,
        'Parent Company': [company_name] * len(new_company_list)
    })

    # Step 3: Calculate the new similarity matrix
    combined_text_companies = pd.concat([company_temp_df['Company'], data_cleaned['Producer']], ignore_index=True)
    similarity_matrix_producers = calculate_similarity_matrix(combined_text_companies, vectorizer)

    # Step 4: Find the highest value for the current index
    current_row_similarity = similarity_matrix_producers[index]
    highest_value = np.max(current_row_similarity)
    highest_index = np.argmax(current_row_similarity)
    highest_matching_company = company_temp_df['Company'].iloc[highest_index]

    # Step 5: Update the original DataFrame
    data_cleaned.at[index, 'Matching Company Highest'] = highest_matching_company
    data_cleaned.at[index, 'Matching Company Highest Value'] = highest_value

    # Step 6: Keep track of the updated entry
    updated_entries.append((index, highest_matching_company))

# Add a new column to the DataFrame to keep track of the updated entries
data_cleaned['Updated Entry'] = 'No'
for index, company in updated_entries:
    data_cleaned.at[index, 'Updated Entry'] = 'Yes, updated to {}'.format(company)

# Save the updated DataFrame
data_cleaned.to_csv('matched_results_updated.csv', index=False)


                                 Producer       Brand  \
4                       Paranapanema S.A.         CbM   
5                       Paranapanema S.A.       CbM-P   
24                        Mantoverde S.A.          MV   
129                            Metalblanc  FMB PB970R   
130                            Metalblanc  FMB PB985R   
159                      Orzel Bialy S.A.  EAGLE 9997   
227  Zaklady Gorniczo-Hutnicze “Boleslaw”      ZGH Z1   
289                                Alucam      ALUCAM   
319          Nordural - Grundartangi ehf.       NA NA   
333        Kombinat Aluminijuma Podgorica         KAP   
334                         Mozal S.A.R.L       MOZAL   
345                        S.C. Alro S.A.        ALRO   
357                         Slovalco a.s.    SLOVALCO   
387            FECS Partecipazioni S.p.A.        FECS   
388                      Raffmetal S.p.A.        R.M.   
390                         Intals S.p.A.          VM   
402                         Alu

SyntaxError: EOL while scanning string literal (<string>, line 4)

In [5]:
import os
import openai
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

# Function to calculate the similarity matrix
def calculate_similarity_matrix(combined_text, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(combined_text)
    vectors_a = tfidf_matrix[:len(company_temp_df)]
    vectors_b = tfidf_matrix[len(company_temp_df):]
    return cosine_similarity(vectors_b, vectors_a)

# Function to simulate API call (replace with your actual API call)
def simulated_api_call(company_name):
    openai.api_key = os.environ.get("OPENAI_API_KEY")


# Prompt for the company name
  
    print(f"Searching matches for {company_name} and its subsidiaries")

# Send the prompt to the API, use this prompt for all values in highest matching
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Data Engineer creating lists to use in an app later. Format the list as a python list always."
            },
            {
                "role": "user",
                "content": f"List all of the subsidiaries, holdings, partners and owners of the company {company_name} as a list in python with no other supporting text. Only the list in a usable format with no line breaks."
            }
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

# Extract the list of subsidiaries from the response
    try:
        subsidiaries = eval(response['choices'][0]['message']['content'])    
    except:
        subsidiaries=["Response Error"]
    print(f"The listed subsidiaries are {subsidiaries}")
    time.sleep(2)
    return subsidiaries

# Read your original data
# ... (Your code to read and preprocess the data)

# Calculate low-value matches
low_value_matches = data_cleaned[data_cleaned['Matching Company Highest Value'] < 0.10]

# Initialize an empty list to keep track of updated entries
updated_entries = []

# Initialize a counter variable
counter = 0

# Loop through each low-value match
for index, row in low_value_matches.iterrows():
    company_name = row['Producer']

    # Step 1: Get the new list from the simulated API call
    new_company_list = simulated_api_call(company_name)

    # Step 2: Create a temporary DataFrame
    company_temp_df = pd.DataFrame({
        'Company': new_company_list,
        'Parent Company': [company_name] * len(new_company_list)
    })

    # Step 3: Calculate the new similarity matrix
    combined_text_companies = pd.concat([company_temp_df['Company'], data_cleaned['Producer']], ignore_index=True)
    similarity_matrix_producers = calculate_similarity_matrix(combined_text_companies, vectorizer)

    # Step 4: Find the highest value for the current index
    current_row_similarity = similarity_matrix_producers[counter]
    highest_value = np.max(current_row_similarity)
    print(highest_value)
    highest_index = np.argmax(current_row_similarity)
    highest_matching_company = company_temp_df['Company'].iloc[highest_index]

    # Step 5: Update the original DataFrame
    data_cleaned.at[index, 'Matching Company Highest'] = highest_matching_company
    data_cleaned.at[index, 'Matching Company Highest Value'] = highest_value

    # Step 6: Keep track of the updated entry
    updated_entries.append((index, highest_matching_company))

    # Increment the counter
    counter += 1

# Add a new column to the DataFrame to keep track of the updated entries
data_cleaned['Updated Entry'] = 'No'
for index, company in updated_entries:
    .+09
    data_cleaned.at[index, 'Updated Entry'] = 'Yes, updated to {}'.format(company)

# Save the updated DataFrame
data_cleaned.to_csv('matched_results_updated.csv', index=False)


Searching matches for Paranapanema S.A. and its subsidiaries
The listed subsidiaries are ['Response Error']
0.0
Searching matches for Mantoverde S.A. and its subsidiaries
The listed subsidiaries are ['Subsidiary A', 'Subsidiary B', 'Subsidiary C', 'Holding A', 'Holding B', 'Partner A', 'Partner B', 'Owner A', 'Owner B']
0.0
Searching matches for Metalblanc and its subsidiaries
The listed subsidiaries are ['Subsidiary A', 'Subsidiary B', 'Subsidiary C', 'Holding X', 'Holding Y', 'Holding Z', 'Partner 1', 'Partner 2', 'Owner 1', 'Owner 2']
0.0
Searching matches for Metalblanc and its subsidiaries
The listed subsidiaries are ['Subsidiary1', 'Subsidiary2', 'Subsidiary3', 'Subsidiary4', 'Holding1', 'Holding2', 'Holding3', 'Holding4', 'Partner1', 'Partner2', 'Partner3', 'Owner1', 'Owner2', 'Owner3']
0.0
Searching matches for Orzel Bialy S.A. and its subsidiaries
The listed subsidiaries are ['Subsidiary 1', 'Subsidiary 2', 'Subsidiary 3', 'Subsidiary 4', 'Holding 1', 'Holding 2', 'Partner 1',

In [25]:
low_value_matches = data_cleaned[data_cleaned['Matching Company Highest Value'] < 0.10]

In [23]:
for index, row in low_value_matches.iterrows():
    print(row['Producer'])
    company_name = row['Producer']
    print(company_name)
print(low_value_matches)

Paranapanema S.A.
Paranapanema S.A.
Paranapanema S.A.
Paranapanema S.A.
Mantoverde S.A.
Mantoverde S.A.
Metalblanc
Metalblanc
Metalblanc
Metalblanc
Orzel Bialy S.A.
Orzel Bialy S.A.
Zaklady Gorniczo-Hutnicze “Boleslaw”
Zaklady Gorniczo-Hutnicze “Boleslaw”
Alucam
Alucam
Nordural - Grundartangi ehf.
Nordural - Grundartangi ehf.
Kombinat Aluminijuma Podgorica
Kombinat Aluminijuma Podgorica
Mozal S.A.R.L
Mozal S.A.R.L
S.C. Alro S.A.
S.C. Alro S.A.
Slovalco a.s.
Slovalco a.s.
FECS Partecipazioni S.p.A.
FECS Partecipazioni S.p.A.
Raffmetal S.p.A.
Raffmetal S.p.A.
Intals S.p.A.
Intals S.p.A.
Alusigma S.A.
Alusigma S.A.
FECS Partecipazioni S.p.A.
FECS Partecipazioni S.p.A.
Raffmetal S.p.A.
Raffmetal S.p.A.
Intals S.p.A.
Intals S.p.A.
Alusigma S.A.
Alusigma S.A.
                                 Producer       Brand  \
4                       Paranapanema S.A.         CbM   
5                       Paranapanema S.A.       CbM-P   
24                        Mantoverde S.A.          MV   
129     